In [1]:
import os

import pandas as pd
import numpy as np

from dotenv import load_dotenv
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
from sklearn.preprocessing import Normalizer
from sklearn.metrics import mean_absolute_error, median_absolute_error

# Grab covid data

In [2]:
dotenv_path = os.path.join(
    os.path.dirname(os.path.abspath('.')),
    '.env'
)
load_dotenv(dotenv_path, verbose=True)
conn_string = os.getenv('DATABASE_URL')
engine = create_engine(conn_string)

In [3]:
%load_ext sql

%sql $conn_string

### Grab all covid state data

In [4]:
sql = """
SELECT f.date_id, f.location_id, cases, recoveries, deaths, 
    cases_100k, testing_rate, hospitalization_rate,
    date, year, month, day_of_week, day_of_month,
    country, state, city, latitude, longitude, population
FROM covid_facts f JOIN date_dim d ON d.date_id = f.date_id
JOIN location_dim l ON l.location_id = f.location_id
WHERE country = 'US' AND city IS NULL
ORDER BY state
"""

us_df = pd.read_sql(sql, engine)

### Remove rows with null population

In [5]:
us_df = us_df.loc[pd.notnull(us_df.population)]

# Add region information for each state

In [6]:
states_df = pd.read_csv('https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv')

In [7]:
states_df = states_df.rename(columns=lambda col: col.lower())

In [8]:
us2_df = us_df.join(states_df.set_index('state'), on='state').sort_values(['state', 'date'])

# Preprocess data
### Normalize case, recoveries, and deaths using the population

In [9]:
us2_df['cases_norm100k'] = us2_df.cases / (us2_df.population / 100_000)
us2_df['recoveries_norm100k'] = us2_df.recoveries / (us2_df.population / 100_000)
us2_df['deaths_norm100k'] = us2_df.deaths / (us2_df.population / 100_000)


### Seperate region per column and remove columns that are not needed

In [10]:
us3_df = pd.get_dummies(us2_df, columns = ['region'])
us3_df = us3_df.drop(['year','month','state code', 'day_of_week', 'longitude', 'division', 'location_id', 'day_of_month', 'city', 'latitude'], axis = 1)

#Define a function that takes each row of states and combines them to one region row.

In [34]:
def flatten_df(df, group_fields):
    grouped = df.groupby(group_fields)
    flattened_df = pd.DataFrame()
    for name, group in grouped:
        row = {}
        row['name'] = name
        row['cases'] = group.cases.sum()
        row['testing_rate'] = group.testing_rate.mean()
        row['cases_norm100k'] = group.cases_norm100k.sum()
        row['recoveries_norm100k'] = group.recoveries_norm100k.sum()
        row['deaths_norm100k'] = group.deaths_norm100k.sum()
        for state in group.state.values:
            state_data =  group[group.state == state]
            row[state+'_cases_norm100k'] = state_data.cases_norm100k.values[0]
            row[state+'_recoveries_norm100k'] = state_data.recoveries_norm100k.values[0]
            row[state+'_deaths_norm100k'] = state_data.deaths_norm100k.values[0]
            row[state+'_testing_rate'] = state_data.testing_rate.values[0]
        flattened_df = flattened_df.append(row, ignore_index = True)
    return flattened_df

# Data preperation: Getting the feature set

## Creates a sliding window that takes 15 days worth of information and creates features based on the 15 days of data.
### returns: 
    X which is the features/predictors
    y which is the actual cases 7 days into the future
    date which holds cases for the current date

In [89]:
def sliding_window(df, segment_time_frame, days_out):
    X = pd.DataFrame()
    y = []
    data_df = pd.DataFrame()
    loop_count = 0
    date = pd.DataFrame()
    date['name'] = df['name']
    date['cases'] = df['cases']
    for index, row in df.drop('name', axis = 1).iterrows():
        loop_count = loop_count + 1
        data_df = data_df.append(row)
        if data_df.shape[0] >= segment_time_frame:
            #Calculate Features
            features = {}
            for column in data_df.columns:
                if column != 'cases':
                    features['Max_' + column] = data_df[column].max()
                    features['Min_' + column] = data_df[column].min()
                    features['AVG_' + column] = data_df[column].mean()
                    if segment_time_frame > 1:
                        features[column + '_2'] = data_df[column].values[-1] - data_df[column].values[-2]
                        if segment_time_frame >= 3:
                            features[column + '_3'] = data_df[column].values[-1] - data_df[column].values[-3]
                            if segment_time_frame >= 5:
                                features[column + '_5'] = data_df[column].values[-1] - data_df[column].values[-5]
                                if segment_time_frame >= 7:
                                    features[column + '_7'] = data_df[column].values[-1] - data_df[column].values[-7]
                                    if segment_time_frame >= 8:
                                        features[column + '_' + str(segment_time_frame)] = data_df[column].values[-1] - data_df[column].values[-segment_time_frame]
            #Append Features
            X = X.append(features, ignore_index = True)
            data_df = data_df.iloc[1:,:]
            try:
                y.append(df.cases[index + days_out])
            except:
                y.append(-1)
    return X.iloc[:, :], y[:], date

## Seperate state by region

In [90]:
us_northeast_df = us3_df[us3_df.region_Northeast == 1].drop(['region_Midwest', 'region_Northeast',  'region_South', 'region_West'], axis = 1)
us_south_df = us3_df[us3_df.region_South== 1].drop(['region_Midwest', 'region_Northeast',  'region_South', 'region_West'], axis = 1)
us_midwest_df = us3_df[us3_df.region_Midwest == 1].drop(['region_Midwest', 'region_Northeast',  'region_South', 'region_West'], axis = 1)
us_west_df = us3_df[us3_df.region_West == 1].drop(['region_Midwest', 'region_Northeast',  'region_South', 'region_West'], axis = 1)

## Flatten each data frame to get region data

In [91]:
northeast_flattened_df = flatten_df(us_northeast_df, ['date'])
south_flattened_df = flatten_df(us_south_df, ['date'])
midwest_flattened_df = flatten_df(us_midwest_df, ['date'])
west_flattened_df = flatten_df(us_west_df, ['date'])

## Get the feature set and fill null values with average

In [92]:
northeast_X, northeast_y, northeast_date = sliding_window(northeast_flattened_df, 15, 7)
northeast_X['Target'] = northeast_y
northeast_X = northeast_X.fillna(northeast_X.mean())

south_X, south_y, south_date = sliding_window(south_flattened_df, 15, 7)
south_X['Target'] = south_y
south_X = south_X.fillna(south_X.mean())

midwest_X, midwest_y, midwest_date = sliding_window(midwest_flattened_df, 15, 7)
midwest_X['Target'] = midwest_y
midwest_X = midwest_X.fillna(midwest_X.mean())

west_X, west_y, west_date = sliding_window(west_flattened_df, 15, 7)
west_X['Target'] = west_y
west_X = west_X.fillna(west_X.mean())


## Split into training and prediction set, Reduce columns based on Chi2 and correlation values, and normalize the features

In [93]:
NORTHEAST_CHOSEN_COLUMNS = ['AVG_Massachusetts_testing_rate', 'AVG_Rhode Island_testing_rate', 'Connecticut_testing_rate_15', 'Massachusetts_testing_rate_15', 'Max_Rhode Island_testing_rate', 'Min_Rhode Island_testing_rate', 'Min_cases_norm100k', 'Vermont_testing_rate_15', 'cases_norm100k_15', 'testing_rate_15']
northeast_train, northeast_pred = northeast_X[northeast_X.Target != -1], northeast_X[northeast_X.Target == -1]
northeast_X_train, northeast_X_pred, northeast_y_train, northeast_y_pred = northeast_train.drop('Target', axis = 1), northeast_pred.drop('Target', axis = 1), northeast_train['Target'], northeast_pred['Target']
northeast_selected_X_train = northeast_X_train[NORTHEAST_CHOSEN_COLUMNS]
northeast_selected_X_pred = northeast_X_pred[NORTHEAST_CHOSEN_COLUMNS]
northeast_transformed_X_train = pd.DataFrame(Normalizer().fit_transform(northeast_selected_X_train), columns = northeast_selected_X_train.columns)
northeast_transformed_X_pred = pd.DataFrame(Normalizer().transform(northeast_selected_X_pred), columns = northeast_selected_X_pred.columns)

In [94]:
SOUTH_CHOSEN_COLUMNS = ['Florida_testing_rate_15', 'Kentucky_testing_rate_15', 'Maryland_testing_rate_15', 'Min_District of Columbia_testing_rate', 'Min_cases_norm100k', 'North Carolina_testing_rate_15', 'West Virginia_testing_rate_15', 'cases_norm100k_15']
south_train, south_pred = south_X[south_X.Target != -1], south_X[south_X.Target == -1]
south_X_train, south_X_pred, south_y_train, south_y_pred = south_train.drop('Target', axis = 1), south_pred.drop('Target', axis = 1), south_train['Target'], south_pred['Target']
south_selected_X_train = south_X_train[SOUTH_CHOSEN_COLUMNS]
south_selected_X_pred = south_X_pred[SOUTH_CHOSEN_COLUMNS]
south_transformed_X_train = pd.DataFrame(Normalizer().fit_transform(south_selected_X_train), columns = south_selected_X_train.columns)
south_transformed_X_pred = pd.DataFrame(Normalizer().transform(south_selected_X_pred), columns = south_selected_X_pred.columns)

In [95]:
MIDWEST_CHOSEN_COLUMNS = ['Indiana_testing_rate_15', 'Max_North Dakota_testing_rate', 'Min_Illinois_testing_rate', 'Min_North Dakota_testing_rate', 'Minnesota_testing_rate_15', 'Missouri_testing_rate_15', 'North Dakota_testing_rate_7', 'Ohio_testing_rate_15', 'Wisconsin_testing_rate_15', 'testing_rate_15']
midwest_train, midwest_pred = midwest_X[midwest_X.Target != -1], midwest_X[midwest_X.Target == -1]
midwest_X_train, midwest_X_pred, midwest_y_train, midwest_y_pred = midwest_train.drop('Target', axis = 1), midwest_pred.drop('Target', axis = 1), midwest_train['Target'], midwest_pred['Target']
midwest_selected_X_train = midwest_X_train[MIDWEST_CHOSEN_COLUMNS]
midwest_selected_X_pred = midwest_X_pred[MIDWEST_CHOSEN_COLUMNS]
midwest_transformed_X_train = pd.DataFrame(Normalizer().fit_transform(midwest_selected_X_train), columns = midwest_selected_X_train.columns)
midwest_transformed_X_pred = pd.DataFrame(Normalizer().transform(midwest_selected_X_pred), columns = midwest_selected_X_pred.columns)

In [96]:
WEST_CHOSEN_COLUMNS = ['Alaska_testing_rate_15', 'California_testing_rate_15', 'Colorado_testing_rate_15', 'Max_Alaska_testing_rate', 'Max_New Mexico_testing_rate', 'Max_Utah_testing_rate', 'Min_California_testing_rate', 'Montana_testing_rate_15', 'Nevada_testing_rate_15', 'Wyoming_testing_rate_15']
west_train, west_pred = west_X[west_X.Target != -1], west_X[west_X.Target == -1]
west_X_train, west_X_pred, west_y_train, west_y_pred = west_train.drop('Target', axis = 1), west_pred.drop('Target', axis = 1), west_train['Target'], west_pred['Target']
west_selected_X_train = west_X_train[WEST_CHOSEN_COLUMNS]
west_selected_X_pred = west_X_pred[WEST_CHOSEN_COLUMNS]
west_transformed_X_train = pd.DataFrame(Normalizer().fit_transform(west_selected_X_train), columns = west_selected_X_train.columns)
west_transformed_X_pred = pd.DataFrame(Normalizer().transform(west_selected_X_pred), columns = west_selected_X_pred.columns)

# Create Models and Get Feature Coefficients

In [97]:
northeast_reg = linear_model.LinearRegression().fit(northeast_transformed_X_train, northeast_y_train)
coef = {}
for idx, name in enumerate(northeast_transformed_X_train.columns):
    coef[name] = northeast_reg.coef_[idx]
print(coef)

{'AVG_Massachusetts_testing_rate': 1354822.6836440896, 'AVG_Rhode Island_testing_rate': -289407.7362192827, 'Connecticut_testing_rate_15': -1413016.3729377198, 'Massachusetts_testing_rate_15': -1618632.8972917062, 'Max_Rhode Island_testing_rate': -697769.43944101, 'Min_Rhode Island_testing_rate': -66517.04228042983, 'Min_cases_norm100k': -1876570.8840820524, 'Vermont_testing_rate_15': 495740.70309168106, 'cases_norm100k_15': 1991731.975432865, 'testing_rate_15': -1477686.358171992}


In [98]:
south_reg = linear_model.LinearRegression().fit(south_transformed_X_train, south_y_train)
coef = {}
for idx, name in enumerate(south_transformed_X_train.columns):
    coef[name] = south_reg.coef_[idx]
print(coef)

{'Florida_testing_rate_15': 1528563.8127750708, 'Kentucky_testing_rate_15': 3847387.1023170445, 'Maryland_testing_rate_15': 1330512.5780275983, 'Min_District of Columbia_testing_rate': 15277477.752419276, 'Min_cases_norm100k': -7683948.55711173, 'North Carolina_testing_rate_15': -8284364.23326627, 'West Virginia_testing_rate_15': 1128163.8728087465, 'cases_norm100k_15': 2503863.434444497}


In [99]:
midwest_reg = linear_model.LinearRegression().fit(midwest_transformed_X_train, midwest_y_train)
coef = {}
for idx, name in enumerate(midwest_transformed_X_train.columns):
    coef[name] = midwest_reg.coef_[idx]
print(coef)

{'Indiana_testing_rate_15': -156207.2864877961, 'Max_North Dakota_testing_rate': -10914076.690230891, 'Min_Illinois_testing_rate': -6638046.633458029, 'Min_North Dakota_testing_rate': -3659119.0108886114, 'Minnesota_testing_rate_15': -3532713.745818762, 'Missouri_testing_rate_15': -10729519.373115787, 'North Dakota_testing_rate_7': -4007128.108816498, 'Ohio_testing_rate_15': -24151673.259564348, 'Wisconsin_testing_rate_15': -696199.4672842445, 'testing_rate_15': 2557791.6290436666}


In [100]:
west_reg = linear_model.LinearRegression().fit(west_transformed_X_train, west_y_train)
coef = {}
for idx, name in enumerate(west_transformed_X_train.columns):
    coef[name] = west_reg.coef_[idx]
print(coef)

{'Alaska_testing_rate_15': -2459464.311345873, 'California_testing_rate_15': -2844465.7315095873, 'Colorado_testing_rate_15': -929936.3925986027, 'Max_Alaska_testing_rate': 5292496.959012461, 'Max_New Mexico_testing_rate': -4282144.657340931, 'Max_Utah_testing_rate': -606961.4417109918, 'Min_California_testing_rate': -3378932.3902869658, 'Montana_testing_rate_15': 2110240.5852365135, 'Nevada_testing_rate_15': 2513296.956759238, 'Wyoming_testing_rate_15': 1034017.2607235867}


# Get model predictions using the pred data and print out some model performance measures explained on Adam's blog
ref https://stackabuse.com/using-machine-learning-to-predict-the-weather-part-2/

In [101]:
northeast_prediction = northeast_reg.predict(northeast_transformed_X_pred)
print("The Explained Variance: %.2f" % northeast_reg.score(northeast_transformed_X_pred, northeast_y_pred))
print("The Mean Absolute Error: %.2f cases" % mean_absolute_error(northeast_y_pred, northeast_prediction))
print("The Median Absolute Error: %.2f cases" % median_absolute_error(northeast_y_pred, northeast_prediction))
print(northeast_prediction)

The Explained Variance: 0.00
The Mean Absolute Error: 1491425.41 cases
The Median Absolute Error: 1491807.80 cases
[1485598.75058569 1489838.47951334 1496269.07135185 1491806.79927414
 1493571.31473748 1496471.96568086 1486414.49884509]


In [102]:
south_prediction = south_reg.predict(south_transformed_X_pred)
print("The Explained Variance: %.2f" % south_reg.score(south_transformed_X_pred, south_y_pred))
print("The Mean Absolute Error: %.2f cases" % mean_absolute_error(south_y_pred, south_prediction))
print("The Median Absolute Error: %.2f cases" % median_absolute_error(south_y_pred, south_prediction))
print(south_prediction)

The Explained Variance: 0.00
The Mean Absolute Error: 4500377.69 cases
The Median Absolute Error: 4486588.33 cases
[4452411.51410636 4426249.27488373 4486587.3268437  4525371.32048392
 4540747.61634873 4478560.13785672 4592709.62670735]


In [103]:
midwest_prediction = midwest_reg.predict(midwest_transformed_X_pred)
print("The Explained Variance: %.2f" % midwest_reg.score(midwest_transformed_X_pred, midwest_y_pred))
print("The Mean Absolute Error: %.2f cases" % mean_absolute_error(midwest_y_pred, midwest_prediction))
print("The Median Absolute Error: %.2f cases" % median_absolute_error(midwest_y_pred, midwest_prediction))
print(midwest_prediction)

The Explained Variance: 0.00
The Mean Absolute Error: 2552495.41 cases
The Median Absolute Error: 2538630.55 cases
[2516314.30227803 2518747.24617815 2538629.55495597 2506855.89164359
 2586885.42015274 2626273.73216125 2573754.71406075]


In [104]:
west_prediction = west_reg.predict(west_transformed_X_pred)
print("The Explained Variance: %.2f" % west_reg.score(west_transformed_X_pred, west_y_pred))
print("The Mean Absolute Error: %.2f cases" % mean_absolute_error(west_y_pred, west_prediction))
print("The Median Absolute Error: %.2f cases" % median_absolute_error(west_y_pred, west_prediction))
print(west_prediction)

The Explained Variance: 0.00
The Mean Absolute Error: 2590351.98 cases
The Median Absolute Error: 2633004.94 cases
[2622454.76934103 2641555.35210671 2640240.0156282  2633003.94411836
 2642001.41129701 2471076.35479093 2482125.03671604]


# Append Predictions to actual cases and months

In [146]:
def appendPredictions(df, predictions):
    for value in predictions:
        new_day = (df.iloc[-1,0] + pd.Timedelta(days = 1))
        series = pd.Series()
        series['name'] = new_day
        series['cases'] = value
        df = df.append(series, ignore_index=True)
    return df

In [147]:
print(northeast_date.shape)
northeast_date_complete = appendPredictions(northeast_date, northeast_prediction)
print(northeast_date_complete.shape)
south_date_complete = appendPredictions(south_date, south_prediction)
midwest_date_complete = appendPredictions(midwest_date, midwest_prediction)
west_date_complete = appendPredictions(west_date, west_prediction)


(255, 2)
(262, 2)


# Create Region Data

In [152]:
region_df = pd.DataFrame()
region_df['date'] = northeast_date_complete.name
region_df['northeast_cases'] = northeast_date_complete.cases
region_df['south_cases'] = south_date_complete.cases
region_df['midwest_cases'] = midwest_date_complete.cases
region_df['west_cases'] = west_date_complete.cases
region_df.tail(10)

,date,northeast_cases,south_cases,midwest_cases,west_cases
252,2020-12-08,2.056180e+06,5.893050e+06,3640667.0,2659078.0
253,2020-12-09,2.091064e+06,5.967068e+06,3696722.0,2691771.0
254,2020-12-10,2.128356e+06,6.041786e+06,3756122.0,2723408.0
255,2020-12-11,1.485599e+06,4.452412e+06,3821649.0,2768534.0
256,2020-12-12,1.489838e+06,4.426249e+06,3878913.0,2811724.0
257,2020-12-13,1.496269e+06,4.486587e+06,3917000.0,2860139.0
258,2020-12-14,1.491807e+06,4.525371e+06,3969831.0,2911305.0
259,2020-12-15,1.493571e+06,4.540748e+06,4030940.0,2966201.0
260,2020-12-16,1.496472e+06,4.478560e+06,4084569.0,3015589.0
261,2020-12-17,1.486414e+06,4.592710e+06,4138167.0,3063466.0
